In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('categorized_particles.csv')
df.type.values.reshape(-1, 1).shape

(114708, 1)

In [10]:
X=df.drop(['type','step','experiment'],axis=1).values #'step','experiment','particle_num','particle_id','frame_num','time_stamp_s', 'x_left','x_right','y_top','y_bottom','edge_particle'],axis=1).values
Y=df['type'].values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 42)

In [55]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

rfc = RandomForestClassifier(criterion = 'gini', random_state = 0) #42) #'entropy' for shannon entropy to minimize log loss (for classification)
rfc.fit(X_train, Y_train)

# Evaluating on Training set
rfc_pred_test = rfc.predict(X_test)
#print('Training Set Evaluation F1-Score=>',f1_score(Y_train,rfc_pred_train,average='micro'))
#rfc_cv_score = cross_val_score(rfc, X_test, Y_test, cv=10, scoring='roc_auc')

score_train=rfc.score(X_train, Y_train) #, sample_weight=None)
score_test=rfc.score(X_test, Y_test) #, sample_weight=None)



print("=== Confusion Matrix ===")
print(confusion_matrix(Y_test, rfc_pred_test))
print('\n')
print("=== Classification Report ===")
print(classification_report(Y_test, rfc_pred_test))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())



=== Confusion Matrix ===
[[    3     4     1     6     0     3     0     0     0     7     0     0]
 [    0 12314     0     1     1     2     0     7     0     0     0     0]
 [    0     4     0     3     0     1     0     1     0     0     0     0]
 [    0     2     0  1544     2     2     0     2     0     2     0     0]
 [    0     4     0    15   886     0     0    11     0     0     0     0]
 [    0     0     0     3     3   102     0     1     0     1     0     0]
 [    0     0     0     0     0     0   519     0     0     0     0     0]
 [    0     5     0     2    22     0     0 17486     0     0     0     0]
 [    0     0     0     0     8     0     0     5     0     0     0     0]
 [    1     0     0     2     0     0     0     0     0  1411     0     2]
 [    0     0     0     0     0     0     0     0     0     3     0     0]
 [    0     0     0     2     0     1     0     0     0     4     0     2]]


=== Classification Report ===


/Applications/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Applications/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                       precision    recall  f1-score   support

         dense fibral       0.75      0.12      0.21        24
       dense globular       1.00      1.00      1.00     12325
      dense ring-like       0.00      0.00      0.00         9
                glass       0.98      0.99      0.99      1554
         multi si oil       0.96      0.97      0.96       916
              protein       0.92      0.93      0.92       110
      schlieren lines       1.00      1.00      1.00       519
         silicone oil       1.00      1.00      1.00     17515
    silicone oil agg.       0.00      0.00      0.00        13
   translucent fibral       0.99      1.00      0.99      1416
 translucent globular       0.00      0.00      0.00         3
translucent ring-like       0.50      0.22      0.31         9

             accuracy                           1.00     34413
            macro avg       0.67      0.60      0.62     34413
         weighted avg       0.99      1.00      1.00 

/Applications/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [56]:
print(score_train) #
print(score_test)

1.0
0.9957574172551071


In [57]:
rfc_pred_test.size

34413

In [58]:
count=0

for i, estimate in enumerate(rfc_pred_test):
    if rfc_pred_test[i]==Y_test[i]:
        count=count+1
    else:
        count=count
print((count/len(rfc_pred_test))*100)

99.57574172551071


In [ ]:
magic functions: timeit # to calculate computing time
%timeit(command)

In [ ]:
#for loop for anything used twice

#function for anything used more than 2 times

In [ ]:
types=df['type'].unique()
grouped_type=df.groupby("type")

list1=[]

for i in range(11):
    list1.append(grouped_type.get_group(types[i]))

for idx, name in enumerate(types):
    for i in range(11):
        list1.append(grouped_type.get_group(types[i]))
    print("number of", name , "=", list1[idx].shape[0])

In [ ]:

fig=plt.figure(figsize=(8,6))
ax1=fig.add_subplot(1,1,1)

colors=['red','blue','green','k','orange','purple','cyan','magenta','lavender','yellow','pink','brown']

#want to replace this for loop with a function that can be input with any number of particle names or indexes

for i in range(len(types)):
    ax1.scatter(list1[i]['max_feret_diam_um'], list1[i]['circularity'], s=0.1, color = colors[i], label=types[i])

ax1.set_xlabel('max feret diamter (um)')
ax1.set_ylabel('circularity')

legend=plt.legend(markerscale=15)

plt.show()